# Import Libraries

In [ ]:
#pip install comet_ml

In [ ]:
# Import comet_ml in the top of your file
#import comet_ml
#from comet_ml import Experiment

#experiment = Experiment(api_key="PXaqoLTlHzWF85kimEv0zHJPz",
 #                       project_name="edsa-movie-recommender", workspace="lejone")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# utilities
import numpy as np
import pandas as pd

#plotting
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline
sns.set()

from sklearn.metrics import mean_squared_error

import surprise
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Context manager allows logging og parameters

#experiment.context_manager("error")

In [ ]:
# URL where experiments can be found

#experiment.url

# Import Data

In [ ]:
train_df = pd.read_csv(r"../input/edsa-recommender-system-predict/train.csv")
test_df =  pd.read_csv(r"../input/edsa-recommender-system-predict/test.csv")
#scores = pd.read_csv(r"../input/edsa-recommender-system-predict/genome_scores.csv")
#tags = pd.read_csv(r"../input/edsa-recommender-system-predict/genome_tags.csv")
#imbd = pd.read_csv(r"../input/edsa-recommender-system-predict/imdb_data.csv")
#links = pd.read_csv(r"../input/edsa-recommender-system-predict/links.csv")
#movies = pd.read_csv(r"../input/edsa-recommender-system-predict/movies.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.shape

# Model-Based Collaborative Filtering

Model-based Collaborative Filtering is based on matrix factorization (MF) which is widely used for recommender systems where it can deal better with scalability and sparsity than Memory-based Collaborative Filtering:

- The goal of MF is to learn the latent preferences of users and the latent attributes of items from known ratings (learn features that describe the characteristics of ratings) to then predict the unknown ratings through the dot product of the latent features of users and items.
- When you have a very sparse matrix, with a lot of dimensions, by doing matrix factorization, you can restructure the user-item matrix into low-rank structure, and you can represent the matrix by the multiplication of two low-rank matrices, where the rows contain the latent vector.

# Support Vector Decomposition (SVD)

A well-known matrix factorization method is Singular value decomposition (SVD). SVD is an algorithm that decomposes a matrix $A$ into the best lower rank (i.e. smaller/simpler) approximation of the original matrix $A$. Mathematically, it decomposes A into a two unitary matrices and a diagonal matrix:

# Model Evaluation¶

Since the data is very big (10 million rows), the train data will be sampled in batches to see how much RMSE score it produces. The data will be trained on three samples of 10 %, 50 % and 80 %. 

In [ ]:
#train_sample_1 = train_df.sample(frac =.1)
#train_sample_5 = train_df.sample(frac =.5)
train_sample_8 = train_df.sample(frac =.8)

In [ ]:
# Use the SVD algorithm.
algo_svd = SVD()

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
#data_1 = Dataset.load_from_df(train_sample_1[['userId', 'movieId', 'rating']], reader)
#data_5 = Dataset.load_from_df(train_sample_5[['userId', 'movieId', 'rating']], reader)
data_8 = Dataset.load_from_df(train_sample_8[['userId', 'movieId', 'rating']], reader)
#data_100 = Dataset.load_from_df(train_sample_100[['userId', 'movieId', 'rating']], reader)

Cross Validation

In [ ]:
# Run 3-fold cross-validation for fourth sample and print results for SVD.
CV_8 = surprise.model_selection.cross_validate(algo_svd, data_8, measures=['RMSE'], cv=3, verbose=True, n_jobs=-1)

Compare Perfomance

In [ ]:
# Calculate RMSE and Test Times

rmse_list = [0.9210, 0.8851, 0.8598]
fit_list = [88.71, 451.35, 721.30]
test_list = [6.36, 34.83, 57.10]

In [ ]:
# plot of Run times
labels = ['Sample 10 %', 'Sample 50 %', 'Sample 80 %', 'Sample 100 %']

x = np.arange(len(labels))  # the label locations
width = 0.15  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, fit_list, width, label='Train')
rects2 = ax.bar(x + width/2, test_list, width, label='Test')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Time (sec)')
ax.set_title('Time To Run Test by Sample Size')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()

In [ ]:
# plot of RMSE values

fig, ax = plt.subplots()

ax.bar(x, rmse_list, align='center')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('RMSE')
ax.set_title('RMSE Over Different Samples ')
ax.set_xticks(x)
ax.set_xticklabels(labels)

plt.xticks(x, labels) #Replace default x-ticks with xs, then replace xs with labels
plt.yticks(rmse_list)

fig.tight_layout()

plt.show()

From the plots above, the following observations can be made:

- The time to train the model increases with the amount of data used to train it. 
- The RMSE score decreases with more samples of the train dataset used.

The model will be fit using the best performing sample (all the data), and generate predictions for this model

In [ ]:
trainset = data_100.build_full_trainset()
algo_svd.fit(trainset)

In [ ]:
# This is for Comet purposes
#pre_comet = []

#for _, row in train_sample.iterrows():
 #   x_unseen = algo_svd.predict(uid=row['userId'],iid=row['movieId'])
  #  pred = x_unseen[3]
   # pre_comet.append(pred)

In [ ]:
# Calculate the predictions
pre = []

for _, row in test_df.iterrows():
    x_unseen = algo_svd.predict(uid=row['userId'],iid=row['movieId'])
    pred = x_unseen[3]
    pre.append(pred)

Format the test data for submission

In [ ]:
test_df['Id'] = test_df['userId'].map(str)+ "_" +test_df['movieId'].map(str)
test_df.head()

In [ ]:
#Sellecting the index of the test dataframe
final_test= test_df["Id"]

In [ ]:
#Creating the submission Dataframe
Final_Table = {'Id': final_test, 'rating':np.round(pre, 1)}
submission = pd.DataFrame(data=Final_Table)
submission = submission[['Id', 'rating']]

In [ ]:
submission.to_csv("Submission.csv",index  = False) #wrting csv file

In [ ]:
# Log metrics

#experiment = Experiment(api_key="PXaqoLTlHzWF85kimEv0zHJPz")
#with experiment.context_manager("error"):
  #algo_svd.fit(trainset)
 # rmse_svd = mean_squared_error(train_sample['rating'],pre_comet, squared=False)
  # returns the validation accuracy
  #experiment.log_metric("rmse", rmse_svd)

In [ ]:
# End the experiment

#experiment.end()

In [ ]:
# Display comet experiment

#experiment.display()

In [ ]:
#import pickle
#model_save_path = "SVD.pkl"
#with open(model_save_path,'wb') as file:
 #   pickle.dump(svd,file)

In [ ]:
#from IPython.display import FileLink
#FileLink(r'SVD.pkl')